In [1]:
# Importamos pandas y le damos el apodo 'pd'
import pandas as pd
import numpy as np # También importamos numpy, es una buena práctica
import ast

print("Pandas importado correctamente.")

Pandas importado correctamente.


In [ ]:
# Cargar el archivo principal de metadata
# (low_memory=False evita un aviso de advertencia sobre tipos de datos mixtos)
df_meta = pd.read_csv('movies_metadata.csv', low_memory=False)

# Cargar el archivo de créditos
df_credits = pd.read_csv('credits.csv')

# Cargar el archivo de keywords
df_keywords = pd.read_csv('keywords.csv')

print("--- Archivos CSV cargados ---")
print(f"Metadata: {df_meta.shape}")
print(f"Credits:  {df_credits.shape}")
print(f"Keywords: {df_keywords.shape}")

--- Archivos CSV cargados ---
Metadata: (45466, 24)
Credits:  (45476, 3)
Keywords: (46419, 2)


In [ ]:
# --- Limpieza de df_meta ---

# 1. Convertir 'id' a numérico. 'errors='coerce'' convierte los IDs malos en 'NaT' (Not a Time) o 'NaN' (Not a Number)
df_meta['id'] = pd.to_numeric(df_meta['id'], errors='coerce')

# 2. Eliminar las filas que tengan esos IDs malos (usualmente son solo 3 filas)
df_meta = df_meta.dropna(subset=['id'])

# 3. Ahora que está limpio, convertir la columna 'id' a tipo 'int' (entero)
df_meta['id'] = df_meta['id'].astype(int)


# --- Limpieza de df_keywords y df_credits ---
# (Estas ya suelen estar limpias, pero nos aseguramos que sean 'int' para la unión)
df_keywords['id'] = df_keywords['id'].astype(int)
df_credits['id'] = df_credits['id'].astype(int)

print("\n--- Limpieza de IDs completada ---")
print(f"Tipo de 'id' en metadata: {df_meta['id'].dtype}")
print(f"Tipo de 'id' en keywords: {df_keywords['id'].dtype}")
print(f"Tipo de 'id' en credits:  {df_credits['id'].dtype}")


--- Limpieza de IDs completada ---
Tipo de 'id' en metadata: int64
Tipo de 'id' en keywords: int64
Tipo de 'id' en credits:  int64


In [ ]:
# 1. Unir metadata con keywords
print("Uniendo metadata y keywords...")
df = df_meta.merge(df_keywords, on='id')

# 2. Unir el resultado anterior con credits
print("Uniendo el resultado con credits...")
df = df.merge(df_credits, on='id')

print("\n¡FUSIÓN COMPLETADA!")

Uniendo metadata y keywords...
Uniendo el resultado con credits...

¡FUSIÓN COMPLETADA!


In [ ]:
# Mostrar la forma (filas, columnas) del DataFrame final
print(f"\nForma del DataFrame final: {df.shape}")

# Mostrar todas las columnas para verificar que 'cast', 'crew', y 'keywords' están
print("\nColumnas del DataFrame unido:")
print(list(df.columns))


Forma del DataFrame final: (46628, 27)

Columnas del DataFrame unido:
['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count', 'keywords', 'cast', 'crew']


In [ ]:
print("Iniciando exportación a CSV...")

# Usamos la función .to_csv()
# 'movies_dataset_unido.csv' es el nombre que le daremos al nuevo archivo.
# index=False es MUY importante. Evita que pandas guarde
# el índice (0, 1, 2, 3...) como una columna extra en tu CSV.

df.to_csv('movies_dataset_unido.csv', index=False)

print("\n¡Éxito! Tu archivo 'movies_dataset_unido.csv' ha sido creado.")

Iniciando exportación a CSV...

¡Éxito! Tu archivo 'movies_dataset_unido.csv' ha sido creado.


In [ ]:
# --- Función 1: El evaluador seguro ---
# Esta función convierte el string '[]' en un objeto lista []
def safe_literal_eval(s):
    try:
        # Intenta convertir el string en un objeto Python (ej. lista)
        return ast.literal_eval(s)
    except (ValueError, SyntaxError, TypeError):
        # Si falla (ej. es un NaN malformado), devuelve una lista vacía
        return []

# --- Función 2: Extraer Nombres (para Genres y Keywords) ---
def get_names(data_list_str):
    # 1. Convertir el string (ej. "[{'name': 'Action'}]") en un objeto lista
    data_list = safe_literal_eval(data_list_str)

    if isinstance(data_list, list):
        names = []
        for item in data_list:
            if 'name' in item:
                # 2. Estandarizar: minúsculas y sin espacios
                clean_name = item['name'].lower().replace(" ", "")
                names.append(clean_name)
        return names
    return []

# --- Función 3: Extraer Top 3 Actores (para Cast) ---
def get_top_cast(cast_list_str, top_n=3):
    cast_list = safe_literal_eval(cast_list_str)

    if isinstance(cast_list, list):
        names = []
        # 3. Limitar a los primeros 'top_n' (3)
        for item in cast_list[:top_n]:
            if 'name' in item:
                # 2. Estandarizar
                clean_name = item['name'].lower().replace(" ", "")
                names.append(clean_name)
        return names
    return []

# --- Función 4: Extraer al Director (para Crew) ---
def get_director(crew_list_str):
    crew_list = safe_literal_eval(crew_list_str)

    if isinstance(crew_list, list):
        # 4. Buscar en toda la lista
        for member in crew_list:
            if member.get('job') == 'Director':
                # 2. Estandarizar
                clean_name = member['name'].lower().replace(" ", "")
                return clean_name
    # Si no se encuentra director o la lista está vacía
    return ""

print("Funciones de 'parsing' creadas exitosamente.")

Funciones de 'parsing' creadas exitosamente.


In [ ]:
print("Aplicando funciones al DataFrame... (Esto puede tardar un momento)")

# 'axis=1' no es necesario aquí, .apply() funciona sobre la Serie
df['genres_list'] = df['genres'].apply(get_names)
df['keywords_list'] = df['keywords'].apply(get_names)
df['cast_list'] = df['cast'].apply(get_top_cast)
df['director'] = df['crew'].apply(get_director) # Esta será un string

print("¡Nuevas columnas creadas!")

# Verifiquemos el resultado
print("\n--- Ejemplo de limpieza (primera película) ---")
print(f"Título: {df.iloc[0]['title']}")
print(f"Géneros Limpios: {df.iloc[0]['genres_list']}")
print(f"Actores Limpios: {df.iloc[0]['cast_list']}")
print(f"Director Limpio: {df.iloc[0]['director']}")

Aplicando funciones al DataFrame... (Esto puede tardar un momento)
¡Nuevas columnas creadas!

--- Ejemplo de limpieza (primera película) ---
Título: Toy Story
Géneros Limpios: ['animation', 'comedy', 'family']
Actores Limpios: ['tomhanks', 'timallen', 'donrickles']
Director Limpio: johnlasseter
